In [11]:
! pip install geopandas pandas sqlalchemy psycopg2-binary openpyxl geoalchemy2 python-dotenv tqdm

/bin/bash: /media/mutakabbir/HDD_2TB_02/Forest_Fire/.venv/bin/pip: /media/mutakabbir/HDD_2TB_01/Forest_Fire/.venv/bin/python: bad interpreter: No such file or directory


In [1]:
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine, TIMESTAMP

import os
import warnings
from tqdm import tqdm
from dotenv import load_dotenv

In [2]:
warnings.filterwarnings("ignore")

In [3]:
climate_ics_date_pattern = '(?P<year>\d{4})(?P<month>\d{2})(?P<day>\d{2})(?P<hour>\d{2})'

In [4]:
climate_dtypes = {
    0: 'object',
    1: 'object',
    2: 'object',
    3: 'int64',
    4: 'int64',
    5: 'object',
    6: 'int64',
    7: 'object',
    8: 'int64',
    9: 'object',
    10: 'int64',
    11: 'object',
    12: 'int64',
    13: 'object',
    14: 'int64',
    15: 'object',
    16: 'int64',
    17: 'object',
    18: 'int64',
    19: 'object',
    20: 'int64',
    21: 'object',
    22: 'int64',
    23: 'object',
    24: 'int64',
    25: 'object',
    26: 'int64',
    27: 'object',
    28: 'int64',
    29: 'object',
    30: 'int64',
    31: 'object',
    32: 'int64',
    33: 'object',
    34: 'int64',
    35: 'object',
    36: 'int64',
    37: 'object',
    38: 'int64',
    39: 'object',
    40: 'int64',
    41: 'object',
    42: 'int64',
    43: 'object',
}

sql_dtypes = {
    "Year Month Day Hour (YYYYMMDDHH)": TIMESTAMP
}

cweeds_processed_sql_dtypes = {
    "date": TIMESTAMP
}

In [5]:
cweeds_flag_map = {
    'Flag': 'Global horizontal irradiance / kJ/m2',
    'Flag.1': 'Direct normal irradiance / kJ/m2',
    'Flag.2': 'Diffuse horizontal irradiance / kJ/m2',
    'Flag.3': 'Global horizontal illuminance / 100 lux',
    'Flag.4': 'Direct normal illuminance / 100 lux',
    'Flag.5': 'Diffuse horizontal illuminance / 100 lux',
    'Flag.6': 'Zenith luminance / 100 Cd/m2',
    'Flag.7': 'Minutes of sunshine / 0-60 minutes',
    'Flag.8': 'Ceiling height / 10 m',
    'Flag.10': 'Visibility / 100 m',
    'Flag.12': 'Station pressure / 10 Pa',
    'Flag.13': 'Dry bulb temperature / 0.1 C',
    'Flag.14': 'Dew point temperature / 0.1 C',
    'Flag.15': 'Wind direction / 0-359 degrees',
    'Flag.16': 'Wind speed / 0.1 m/s',
    'Flag.17': 'Total sky cover / 0-10 in tenths',
    'Flag.18': 'Opaque sky cover / 0-10 in tenths',
    'Flag.19': 'Snow cover (0 = no snow cover 1 = snow cover)'
}

In [6]:
cweeds_digit_code_map = {
    'Sky condition': (4,'Flag.9'),
    'Present Weather': (8, 'Flag.11')
    
}

In [7]:
cweeds_column_name_mapping = {
    'ECCC station identifier': 'climate_id', 
    'timestamp': 'timestamp',
    'Global horizontal irradiance / kJ/m2': 'global_horizontal_irradiance',
    'Direct normal irradiance / kJ/m2': 'direct_noraml_irradiance',
    'Diffuse horizontal irradiance / kJ/m2': 'diffuse_horizontal_irradiance',
    'Global horizontal illuminance / 100 lux': 'global_horizontal_lluminance',
    'Direct normal illuminance / 100 lux': 'direct_noraml_illuminance',
    'Diffuse horizontal illuminance / 100 lux': 'diffused_horizontal_illuminance',
    'Zenith luminance / 100 Cd/m2': 'zeenath_luminance', 
    'Minutes of sunshine / 0-60 minutes': 'sunshine_min',
    'Ceiling height / 10 m': 'ceiling_height', 
    'Visibility / 100 m': 'visibility',
    'Station pressure / 10 Pa': 'pressure', 
    'Dry bulb temperature / 0.1 C': 'dry_bulb_temperature',
    'Dew point temperature / 0.1 C': 'dew_point_temperature', 
    'Wind direction / 0-359 degrees': 'wind_direction',
    'Wind speed / 0.1 m/s': 'wind_speed', 
    'Total sky cover / 0-10 in tenths': 'total_sky_cover',
    'Opaque sky cover / 0-10 in tenths': 'opaque_sky_cover',
    'Snow cover (0 = no snow cover 1 = snow cover)': 'snow', 
    'Sky condition_0': 'sky_condition_0',
    'Sky condition_1': 'sky_condition_1', 
    'Sky condition_2': 'sky_condition_2', 
    'Sky condition_3': 'sky_condition_3',
    'Present Weather_0': 'weather_0', 
    'Present Weather_1': 'weather_1', 
    'Present Weather_2': 'weather_2',
    'Present Weather_3': 'weather_3', 
    'Present Weather_4': 'weather_4', 
    'Present Weather_5': 'weather_5',
    'Present Weather_6': 'weather_6', 
    'Present Weather_7': 'weather_7'
}

In [8]:
cweeds_agg_list = list(cweeds_flag_map.values()) + [f"{col}_{index}" for col in cweeds_digit_code_map for index in range(cweeds_digit_code_map[col][0])]
cweeds_agg_list

['Global horizontal irradiance / kJ/m2',
 'Direct normal irradiance / kJ/m2',
 'Diffuse horizontal irradiance / kJ/m2',
 'Global horizontal illuminance / 100 lux',
 'Direct normal illuminance / 100 lux',
 'Diffuse horizontal illuminance / 100 lux',
 'Zenith luminance / 100 Cd/m2',
 'Minutes of sunshine / 0-60 minutes',
 'Ceiling height / 10 m',
 'Visibility / 100 m',
 'Station pressure / 10 Pa',
 'Dry bulb temperature / 0.1 C',
 'Dew point temperature / 0.1 C',
 'Wind direction / 0-359 degrees',
 'Wind speed / 0.1 m/s',
 'Total sky cover / 0-10 in tenths',
 'Opaque sky cover / 0-10 in tenths',
 'Snow cover (0 = no snow cover 1 = snow cover)',
 'Sky condition_0',
 'Sky condition_1',
 'Sky condition_2',
 'Sky condition_3',
 'Present Weather_0',
 'Present Weather_1',
 'Present Weather_2',
 'Present Weather_3',
 'Present Weather_4',
 'Present Weather_5',
 'Present Weather_6',
 'Present Weather_7']

In [9]:
CWEEDS_STATION_DATA_DIR = "../../data/ics/CWEEDS"
PATH_TO_DOT_ENV = "../../.env"

DATABASE_TYPE = "postgresql"
DATABASE_HOST = "localhost"

CWEEDS_STATION_TABLE_NAME = "W_s"
CWEEDS_STATION_PROCESSED_TABLE_NAME = "W_sp"

In [10]:
load_dotenv(PATH_TO_DOT_ENV)

DATABASE_NAME = os.environ.get("DATABASE_NAME")
POSTGRES_USER = os.environ.get("POSTGRES_USER")
POSTGRES_PASSWORD = os.environ.get("POSTGRES_PASSWORD")
POSTGRES_HOST_PORT = os.environ.get("POSTGRES_HOST_PORT")
POSTGRES_CONTAINER_PORT = os.environ.get("POSTGRES_CONTAINER_PORT")

In [11]:
engine = create_engine(f"{DATABASE_TYPE}://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{DATABASE_HOST}:{POSTGRES_HOST_PORT}/{DATABASE_NAME}")

In [12]:
def handel_null_values(df, map:dict = cweeds_flag_map):
    for flag in map:
        column = map[flag]
        df[column] = df[[column, flag]].apply(
            lambda row: 0 if row[flag] == "9" else row[column],
            axis = 1
        )
    return df

In [13]:
def expand_digit_code(df, map:dict=cweeds_digit_code_map):
    for column in map:
        num_code, flag = map[column]
        new_columns = [f"{column}_{index}" for index in range(num_code)]
        df[new_columns] = df.apply(
            lambda row: list(str(row[column]).zfill(num_code)) if row[flag] != "9" else list(str(0).zfill(num_code)),
            axis = 1,
            result_type ='expand'
        )
        df[new_columns] = df[new_columns].astype(int)
    return df

In [129]:
provience_list = os.listdir(CWEEDS_STATION_DATA_DIR)
for provience_dir in provience_list:
    province_dir_path = f"{CWEEDS_STATION_DATA_DIR}/{provience_dir}"
    provience_station_list = os.listdir(province_dir_path)
    provience_progress_bar = tqdm(
        provience_station_list,
        desc = provience_dir,
    )
    for file_name in provience_progress_bar:
        file_path = f"{province_dir_path}{os.sep}{file_name}"
        # read data to DF
        climate_df = pd.read_csv(file_path, sep=",", skiprows=[0,1,2], header=None, parse_dates=[2], dtype=climate_dtypes)
        climate_header_df = pd.read_csv(file_path, nrows=0, header=2)
        # merging the columns 
        climate_df.drop([44], axis=1, inplace=True)
        climate_df.rename({index:name for index, name in enumerate(climate_header_df.columns)}, axis=1, inplace=True)
        # parse as datatime
        climate_df["Year Month Day Hour (YYYYMMDDHH)"] = pd.to_datetime(climate_df["Year Month Day Hour (YYYYMMDDHH)"].str.extract(climate_ics_date_pattern, expand=True))
        # # handel missing values
        # climate_df = handel_null_values(df = climate_df)
        # # expand digit codes
        # climate_df = expand_digit_code(df = climate_df)
        # # convert hourly reading to daily readings
        # climate_df['timestamp'] = climate_df['Year Month Day Hour (YYYYMMDDHH)'].dt.date
        # climate_df = climate_df[cweeds_agg_list + ['ECCC station identifier', 'timestamp']].groupby(
        #     by = ['ECCC station identifier', 'timestamp']
        # )[cweeds_agg_list].mean().reset_index()
        # # rename columns
        # climate_df.rename(cweeds_column_name_mapping, axis=1, inplace=True)

        try:
            climate_df.to_sql(name=CWEEDS_STATION_TABLE_NAME, con=engine, if_exists='append', index=False, dtype=sql_dtypes)  
        except Exception as e:
            print(f"{file_name}:   {e}")

        # deleted all data 
        del climate_header_df
        del climate_df

        provience_progress_bar.set_postfix_str(file_name)

CWEEDS_2020_NS:   0%|          | 0/29 [00:00<?, ?it/s]


In [10]:
with engine.connect() as con:
    con.execute(f'ALTER TABLE "{CWEEDS_STATION_TABLE_NAME}" ADD PRIMARY KEY ("ECCC station identifier", "Year Month Day Hour (YYYYMMDDHH)");')

In [14]:
pre_process_cweeds_query = """
select 
	ws."ECCC station identifier" as climate_id,
	cast(ws."Year Month Day Hour (YYYYMMDDHH)" as date) as date,
	avg("Extraterrestrial irradiance / kJ/m2") as "extraterrestrial_irradiance",
    avg(
        case 
            when "Flag" = '9' then 0
            else "Global horizontal irradiance / kJ/m2"
        end
    ) as "global_horizontal_irradiance",
    avg(
        case 
            when "Flag.1" = '9' then 0
            else "Direct normal irradiance / kJ/m2"
        end
    ) as "direct_normal_irradience",
    avg(
        case 
            when "Flag.2" = '9' then 0
            else "Diffuse horizontal irradiance / kJ/m2"
        end
    ) as "diffuse_horizontal_irradiance",
    avg(
        case 
            when "Flag.3" = '9' then 0
            else "Global horizontal illuminance / 100 lux" 
        end
    ) / 10 "global_horizontal_illumination_klux", -- calculate /kluxas 
    avg(
        case 
            when "Flag.4" = '9' then 0 
            else "Direct normal illuminance / 100 lux" 
        end
    ) / 10 as "direct_normal_illumination_klux", -- calculate /klux
    avg(
        case 
            when "Flag.5" = '9' then 0 
            else "Diffuse horizontal illuminance / 100 lux"
        end
    ) / 10 as "diffuse_horizontal_illumination_klux", -- calculate /klux
    avg(
        case 
            when "Flag.6" = '9' then 0 
            else "Zenith luminance / 100 Cd/m2"
        end
    ) as "zenith_illumination",
    avg(
        case 
            when "Flag.7" = '9' then 0 
            else "Minutes of sunshine / 0-60 minutes"
        end
    ) as "sunlight_min",
    avg(
        case 
            when "Flag.8" = '9' then 0 
            when "Ceiling height / 10 m" = 7777 then 3000 -- 7777 represents no limit. hence put max value of 3,000 
            else "Ceiling height / 10 m" 
        end
    ) * 10 as "ceiling_height_meters", -- multiply wwith 10 since it is represented per 10 m
    avg(
        case 
            when "Flag.9" = '9' then 0
            else substring(lpad("Sky condition"::text, 4, '0'), 1, 1)::integer  -- extract layer 1
        end
    ) as "sky_layer_1",
    avg(
        case 
            when "Flag.9" = '9' then 0
            else substring(lpad("Sky condition"::text, 4, '0'), 2, 1)::integer -- extract layer 2
        end
    ) as "sky_layer_2",
    avg(
        case 
            when "Flag.9" = '9' then 0
            else substring(lpad("Sky condition"::text, 4, '0'), 3, 1)::integer -- extract layer 3
        end
    ) as "sky_layer_3",
    avg(
        case 
            when "Flag.9" = '9' then 0
            else substring(lpad("Sky condition"::text, 4, '0'), 4, 1)::integer -- extract layer 4
        end
    ) as "sky_layer_4",
    avg(
        case 
            when "Flag.10" = '9' then 0
            else "Visibility / 100 m" 
        end
    ) / 10 as "visibility_km",  -- convert to visibility / 1 km
    avg(
        case 
            when "Flag.11" = '9' then 0
            else substring(lpad("Present Weather"::text, 8, '0'), 1, 1)::integer
        end
    ) as "weather_thunderstorm",
    avg(
        case 
            when "Flag.11" = '9' then 0
            else substring(lpad("Present Weather"::text, 8, '0'), 2, 1)::integer
        end
    ) as "weather_rain",
    avg(
        case 
            when "Flag.11" = '9' then 0
            else substring(lpad("Present Weather"::text, 8, '0'), 3, 1)::integer
        end
    ) as "weather_drizzle",
    avg(
        case 
            when "Flag.11" = '9' then 0
            else substring(lpad("Present Weather"::text, 8, '0'), 4, 1)::integer
        end
    ) as "weather_snow_1",
    avg(
        case 
            when "Flag.11" = '9' then 0
            else substring(lpad("Present Weather"::text, 8, '0'), 5, 1)::integer
        end
    ) as "weather_snow_2",
    avg(
        case 
            when "Flag.11" = '9' then 0
            else substring(lpad("Present Weather"::text, 8, '0'), 6, 1)::integer
        end
    ) as "weather_ice",
    avg(
        case 
            when "Flag.11" = '9' then 0
            else substring(lpad("Present Weather"::text, 8, '0'), 7, 1)::integer
        end
    ) as "weather_visibility_1",
    avg(
        case 
            when "Flag.11" = '9' then 0
            else substring(lpad("Present Weather"::text, 8, '0'), 8, 1)::integer
        end
    ) as "weather_visibility_2",
    avg(
        case 
            when "Flag.12" = '9' then 0
            else "Station pressure / 10 Pa"  
        end
    ) / 100 as "pressure_kpa", -- convert to presure / kPa
    avg(
        case 
            when "Flag.13" = '9' then null
            else "Dry bulb temperature / 0.1 C"
        end
    ) / 10 as "dry_bulb_temp_c", -- convert to / 1 C
    avg(
        case 
            when "Flag.14" = '9' then null
            else "Dew point temperature / 0.1 C"
        end
    ) / 10 as "dew_point_temp_c", -- convert to / 1 C
    avg(
        case 
            when "Flag.15" = '9' then null
            else "Wind direction / 0-359 degrees"
        end
    ) as "wind_direction_deg",
    avg(
        case 
            when "Flag.16" = '9' then null
            else "Wind speed / 0.1 m/s"
        end
    )  / 10 as "wind_speed_mps",
    avg(
        case 
            when "Flag.17" = '9' then null
            else "Total sky cover / 0-10 in tenths"
        end
    ) as "sky_cover",
    avg(
        case 
            when "Flag.18" = '9' then null
            else "Opaque sky cover / 0-10 in tenths"
        end
    ) as "sky_cover_opaque",
    avg(
        case 
            when "Flag.19" = '9' then 0
            else "Snow cover (0 = no snow cover 1 = snow cover)"
        end
    ) as "snow"
from 
	"W_s" ws
group by 
	climate_id, date
"""

In [15]:
preprocessed_cweeds_df = pd.read_sql(
    sql = pre_process_cweeds_query,
    con = engine,
)

In [16]:
preprocessed_cweeds_df.fillna(0, inplace=True)

In [17]:
preprocessed_cweeds_df.to_sql(
    name=CWEEDS_STATION_PROCESSED_TABLE_NAME, 
    con=engine, 
    if_exists='replace', 
    index=False, 
    dtype=cweeds_processed_sql_dtypes) 

93

In [18]:
with engine.connect() as con:
    con.execute(f'ALTER TABLE "{CWEEDS_STATION_PROCESSED_TABLE_NAME}" ADD PRIMARY KEY ("climate_id", "date");')